In [1]:
import sys
from numpy import *

In [2]:
import xap_mcmc_funs as xap

In [3]:
import sherpa.astro.ui as sherpa

## ObsID #1 

In [4]:
# F is the matrix of PSF and Expmap
F1=array([[  8.83838290e-01,   1.95051380e-03,   1.24255146e+01],
          [  2.16125910e-03,   8.83791509e-01,   1.20655538e+01],
          [  6.85424097e-02,   6.31054323e-02,   2.79000000e+02]])

In [5]:
F1[1][0]

0.0021612591

`F1` in the above example  

- `F1[0,0]` is psf-fraction for source 0 in source 0's region, ie ~90%
- `F1[0,1]` is psf-fraction for source 1 in source 0's region.
- `F1[1,0]` is psf-fraction for source 0 in source 1's region.
- `F1[1,1]` is psf-fraction for source 1 in source 1's region, ie ~90%
- `F1[0,2]` is the area of source 0's region.
- `F1[1,2]` is the area of source 1's region.
- `F1[2,0]` is psf-fraction for source 0 in background region.
- `F1[2,1]` is psf-fraction for source 1 in background region.
- `F1[2,2]` is the area of background region.

What about exposure?  The original `xap` multiplies `F*E` where `E` is vector with average exposures.  So that multiplies the area terms too `F1[2,:]` ? 



In [6]:
# C is the counts array
C1=array([4.0,923.0,74.0])

`C1` is the counts array.

- `C1[0]` counts in source region 0
- `C1[1]` counts in source region 1
- `C1[2]` counts in background region


In [7]:
# Index (obsid number, just 0 to N)
ix1=array([0,1,2])

In [8]:
# ???? These are initial values to the model, how to compute?
s1=array([3.1972e0,1.0425e3,3.8828e-2])
sigma_s1=array([sqrt(5.5871e0),sqrt(1.1529e3),sqrt(6.7777e-4)])

Not sure where these are coming from exactly.  I suspect these the the _MLE_ estimates and Gaussian error estimates?


In [9]:
# This comes from xap_funs.py
def get_s_sigma(F,C):
    import numpy as np
    import numpy.linalg as la
    Finv = la.inv(F)
    s = np.dot(Finv,C)
    Finv2 = Finv*Finv
    sigma_s = np.dot(Finv2,C)
    return s,sigma_s

In [10]:
s,sigma_s = get_s_sigma(F1,C1)
print(s)
print(sigma_s)

[  1.81894088e+00   1.04396829e+03   2.86568291e-02]
[  5.34677567e+00   1.18919837e+03   1.02445763e-03]


> Humm, these numbers don't match the `s1` and `sigma_s1` values above, but not too far off ...

In [11]:
s1 = s
sigma_s1 = sqrt(sigma_s)

In [12]:
# Create model
apertures1=xap.Apertures(F1)

In [13]:
help(apertures1)

Help on Apertures in module xap_mcmc_funs object:

class Apertures(builtins.object)
 |  Class of user-defined models to compute model counts in source or
 |  background apertures. Model parameters are source and background
 |  intensities. Data are raw counts in apertures. The vector of model
 |  counts is computed from the vector of intensities by application of
 |  the ECF/Exposure matrix F (see eq. 7 and Table 1 of Primini & Kashyap,
 |  2014, ApJ, 796, 24.)
 |  
 |  Methods:
 |  __init__(self,F)
 |    define F matrix
 |  __call__(self, params,iap)
 |    compute model counts for vector aperture for F and model intensities given
 |    in params array.
 |  Attributes:
 |    Print F.
 |  
 |  Methods defined here:
 |  
 |  __call__(self, params, iap)
 |      Call self as a function.
 |  
 |  __init__(self, F)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  attributes(self)
 |  
 |  ----------------------------------------------------------------------
 |

## ObsID #2


In [14]:
F2=array([[  8.83838290e-01,   1.95051380e-03,   1.24255146e+01],
          [  2.16125910e-03,   8.83791509e-01,   1.20655538e+01],
          [  6.85424097e-02,   6.31054323e-02,   2.79000000e+02]])
C2=array([11.0,881.0,78.0])
ix2=array([0,1,2])
s2=array([1.0558e+01,9.9642e+02,5.7549e-02])
sigma_s2=array([sqrt(1.5597e+01),sqrt(1.1215e+03),sqrt(9.4478e-04)])
apertures2=xap.Apertures(F2)


In [15]:
s,sigma_s = get_s_sigma(F2,C2)
s2 = s
sigma_s2 = sqrt(sigma_s)
print(s)
print(sigma_s)

[  9.51740878e+00   9.96109250e+02   5.19274158e-02]
[  1.43801231e+01   1.13510313e+03   1.07429346e-03]


## Create sherpa models


In [16]:
sherpa.load_user_model(apertures1,"ap1")

In [17]:
sherpa.load_user_model(apertures2,"ap2")

In [18]:
sherpa.add_user_pars("ap1",['s0_1','s1_1','b_1'],s1,parmins=s1-5.0*sigma_s1,parmaxs=s1+5.0*sigma_s1)
sherpa.add_user_pars("ap2",['s0_2','s1_2','b_2'],s2,parmins=s2-5.0*sigma_s2,parmaxs=s2+5.0*sigma_s2)

In [19]:
sherpa.set_model(1,ap1)
sherpa.set_model(2,ap2)

In [20]:
sherpa.load_arrays(1,ix1,C1,sherpa.Data1D)
sherpa.load_arrays(2,ix2,C2,sherpa.Data1D)

In [21]:
print(sherpa.get_model(1))

usermodel.ap1
   Param        Type          Value          Min          Max      Units
   -----        ----          -----          ---          ---      -----
   ap1.s0_1     thawed      1.81894     -9.74261      13.3805           
   ap1.s1_1     thawed      1043.97      871.545      1216.39           
   ap1.b_1      thawed    0.0286568    -0.131379     0.188693           


In [22]:
print(sherpa.get_model(2))

usermodel.ap2
   Param        Type          Value          Min          Max      Units
   -----        ----          -----          ---          ---      -----
   ap2.s0_2     thawed      9.51741     -9.44316       28.478           
   ap2.s1_2     thawed      996.109      827.653      1164.57           
   ap2.b_2      thawed    0.0519274    -0.111955     0.215809           


## Link parameters


In [23]:
# Source is contant across observations
ap1.s0_1=ap2.s0_2
ap1.s1_1=ap2.s1_2

In [24]:
print(sherpa.get_model(1))
print(sherpa.get_model(2))

usermodel.ap1
   Param        Type          Value          Min          Max      Units
   -----        ----          -----          ---          ---      -----
   ap1.s0_1     linked      9.51741           expr: ap2.s0_2           
   ap1.s1_1     linked      996.109           expr: ap2.s1_2           
   ap1.b_1      thawed    0.0286568    -0.131379     0.188693           
usermodel.ap2
   Param        Type          Value          Min          Max      Units
   -----        ----          -----          ---          ---      -----
   ap2.s0_2     thawed      9.51741     -9.44316       28.478           
   ap2.s1_2     thawed      996.109      827.653      1164.57           
   ap2.b_2      thawed    0.0519274    -0.111955     0.215809           


## Fit it


In [25]:
sherpa.set_stat("cash")
sherpa.set_method("moncar")
sherpa.fit(1,2)

Datasets              = 1, 2
Method                = moncar
Statistic             = cash
Initial fit statistic = -21982.9
Final fit statistic   = -21986.4 at function evaluation 819
Data points           = 6
Degrees of freedom    = 2
Change in statistic   = 3.46364
   ap1.b_1        0.0281873   
   ap2.s0_2       5.58633     
   ap2.s1_2       1020.04     
   ap2.b_2        0.0529076   


In [26]:
sherpa.covar()

Datasets              = 1, 2
Confidence Method     = covariance
Iterative Fit Method  = None
Fitting Method        = moncar
Statistic             = cash
covariance 1-sigma (68.2689%) bounds:
   Param            Best-Fit  Lower Bound  Upper Bound
   -----            --------  -----------  -----------
   ap1.b_1         0.0281873   -0.0307121    0.0307121
   ap2.s0_2          5.58633     -2.22404      2.22404
   ap2.s1_2          1020.04     -24.1055      24.1055
   ap2.b_2         0.0529076   -0.0327194    0.0327194


## Generate draws

In [27]:
stats,accept,params=sherpa.get_draws(niter=10000)

Using Priors:
ap1.b_1: <function flat at 0x7fd7af6a1b70>
ap2.s0_2: <function flat at 0x7fd7af6a1b70>
ap2.s1_2: <function flat at 0x7fd7af6a1b70>
ap2.b_2: <function flat at 0x7fd7af6a1b70>


## Write output

In [28]:
xap.write_draws("sim_1and2_draws.fits",['b1','s0','s1','b2'],stats,accept,params)

In [29]:
!dmlist sim_1and2_draws.fits blocks,cols

 
--------------------------------------------------------------------------------
Dataset: sim_1and2_draws.fits
--------------------------------------------------------------------------------
 
     Block Name                          Type         Dimensions
--------------------------------------------------------------------------------
Block    1: PRIMARY                        Null        
Block    2: Draws                          Table         6 cols x 10001    rows
 
--------------------------------------------------------------------------------
Columns for Table Block Draws
--------------------------------------------------------------------------------
 
ColNo  Name                 Unit        Type             Range
   1   Stat                              Real8          -Inf:+Inf            
   2   Accept                            Logical                             
   3   b1                                Real8          -Inf:+Inf            
   4   s0                    